# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [2]:
# Load the name mapping for the column names
# Specifiy the local directory
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
# sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [3]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
dirpath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-4/ctd/'

In [4]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [5]:
# Now write a function to autopopulate the bottle summary sample sheet
sbe_name_map;
dirpath;

# Now 
files = [x for x in os.listdir(dirpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(dirpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast #'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(dirpath+outname)

In [6]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(dirpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(dirpath+file))
    else:
        pass

In [7]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [8]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [9]:
df.rename(columns={'Bottle Position':'Niskin #'},inplace=True)
df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x) ) )

In [11]:
df.columns.values

array(['Unnamed: 0', 'Niskin #', 'Date Time', 'Pressure, Digiquartz [db]',
       'Depth [salt water, m]', 'Latitude [deg]', 'Longitude [deg]',
       'Temperature [ITS-90, deg C]', 'Temperature, 2 [ITS-90, deg C]',
       'Conductivity [S/m]', 'Conductivity, 2 [S/m]',
       'Salinity, Practical [PSU]', 'Salinity, Practical, 2 [PSU]',
       'Oxygen, SBE 43 [ml/l]',
       'Oxygen Saturation, Garcia & Gordon [ml/l]',
       'Beam Attenuation, WET Labs C-Star [1/m]',
       'Beam Transmission, WET Labs C-Star [%]', 'Hex name',
       'Start Latitude', 'Start Longitude', 'Start Time', 'Cast #'],
      dtype=object)

In [12]:
df.drop(columns='Unnamed: 0',inplace=True)

In [13]:
df.to_csv(dirpath+'Irminger-4_Summary.csv')

In [14]:
os.listdir(dirpath)

['AR21_003.bl',
 'AR21_007.XMLCON',
 'AR21_011.hex',
 'AR21 Deck Test1.bl',
 'AR21 Deck Test1.hdr',
 'AR21 Deck Test1.hex',
 'AR21 DECK TEST1.XMLCON',
 'AR21 Deck Test2.bl',
 'AR21 Deck Test2.hdr',
 'AR21 Deck Test2.hex',
 'AR21 DECK TEST2.XMLCON',
 'ar21.xmlcon',
 'ar21999.bl',
 'ar21999.btl',
 'ar21999.hdr',
 'ar21999.hex',
 'ar21999.ros',
 'ar21999.sum',
 'AR21999.XMLCON',
 'AR21_001.bl',
 'AR21_001.hdr',
 'AR21_001.hex',
 'AR21_001.XMLCON',
 'AR21_002.bl',
 'AR21_002.hdr',
 'AR21_002.hex',
 'AR21_002.XMLCON',
 'AR21_003.btl',
 'AR21_003.hdr',
 'AR21_003.hex',
 'AR21_003.ros',
 'AR21_003.sum',
 'AR21_003.XMLCON',
 'AR21_004.bl',
 'AR21_004.btl',
 'AR21_004.hdr',
 'AR21_004.hex',
 'AR21_004.ros',
 'AR21_004.sum',
 'AR21_004.XMLCON',
 'AR21_005.bl',
 'AR21_005.btl',
 'AR21_005.hdr',
 'AR21_005.hex',
 'AR21_005.ros',
 'AR21_005.sum',
 'AR21_005.XMLCON',
 'AR21_006.bl',
 'AR21_006.btl',
 'AR21_006.hdr',
 'AR21_006.hex',
 'AR21_006.ros',
 'AR21_006.sum',
 'AR21_006.XMLCON',
 'AR21_007.bl

### Oxygen & Salinity 
Now, we need to add the 

In [15]:
# Need a way to map the oxygen and salinity samples to the CTD samples
# Load the CTD sample logs

In [23]:
botpath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-4/Water Sampling/Irminger_Sea-04_AR21_CTD_Sampling_Log.xlsx'
sample_log = pd.read_excel(botpath,sheet_name='Summary',header=0)

In [24]:
sample_log.head()

,Cast #,Niskin #,Time,Target Depth,Trip Depth,Potential Temp,Salinity,Oxygen Bottle #,Ph Bottle #,DIC/TA Bottle #,Salts Bottle #,Nitrate Bottle 1,Chlorophyll Brown Bottle #,Chlorophyll Filter Sample #,Unnamed: 14,Chlorophyll Brown Bottle Volume,Chlorophyll LN Tube,Comments
0,6,1.0,NaN,NaN,1000.0,NaN,34.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,2.0,NaN,NaN,800.0,NaN,34.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTD O2 ~4.76
2,6,3.0,NaN,NaN,800.0,NaN,34.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,4.0,NaN,NaN,300.0,NaN,34.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTD O2 ~4.56
4,6,5.0,NaN,NaN,300.0,NaN,34.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Rename the column headers
sample_log.rename(columns=lambda x: 'Sample Log: ' + x.strip(), inplace=True)
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: str(x) )
sample_log['Sample Log: Cast #'] = sample_log['Sample Log: Cast #'].apply(lambda x: str(x).zfill(3))

In [26]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-4/'

In [27]:
# Load the oxygen data
sal_dir = basepath+'Water Sampling/Irminger_Sea-04_AR21_Oxygen_and_Salinity_Sample_Data/'
sal_df = pd.DataFrame()
oxy_df = pd.DataFrame()

for file in os.listdir(sal_dir):
    if 'SAL' in file:
        sal_df = sal_df.append(pd.read_excel(sal_dir+file))
    elif 'OXY' in file:
        oxy_df = oxy_df.append(pd.read_excel(sal_dir+file))

In [28]:
sal_df.rename(columns=lambda x: 'SAL: ' + x.strip(), inplace=True)
sal_df.dropna(subset=['SAL: Cruise'], inplace=True)
sal_df.fillna(-999, inplace=True)
sal_df['SAL: Station'] = sal_df['SAL: Station'].apply(lambda x: str( int(x) ).zfill(3) )
sal_df['SAL: Niskin #'] = sal_df['SAL: Niskin #'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Sample'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Case ID'] + sal_df['SAL: Sample']

In [29]:
oxy_df.rename(columns=lambda x: 'OXY: ' + x.strip(), inplace=True)
oxy_df.dropna(subset=['OXY: Cruise'], inplace=True)
oxy_df.fillna(-999, inplace=True)
oxy_df['OXY: Station'] = oxy_df['OXY: Station'].apply(lambda x: str( int(x) ).zfill(3) )
oxy_df['OXY: Niskin #'] = oxy_df['OXY: Niskin #'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Sample#'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Case'] + oxy_df['OXY: Sample#']

In [31]:
# Now I need to merge dataframes together
sample_log = sample_log.merge(sal_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Salts Bottle #'], right_on=['SAL: Station','SAL: Sample'])

In [32]:
sample_log = sample_log.merge(oxy_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Oxygen Bottle #'],
                              right_on=['OXY: Station','OXY: Sample#'])

In [33]:
# Now I need to join the 
sample_log[['Sample Log: Oxygen Bottle #', 'OXY: Sample#']].dropna()

,Sample Log: Oxygen Bottle #,OXY: Sample#
5,T1,T1
6,T2,T2
7,T3,T3
8,T4,T4
9,T5,T5
10,T6,T6
11,T7,T7
12,T8,T8
13,T9,T9
14,T10,T10


In [40]:
df['Cast #'] = df['Cast #'].apply(lambda x: str(x).zfill(3) )

In [42]:
# Now, merge the oxygen and salinity data into the bottle summary file
result = df.merge(sample_log[['Sample Log: Cast #','Sample Log: Niskin #','SAL: Salinity','OXY: Oxy']],
                  how='left', left_on=['Cast #','Niskin #'], right_on=['Sample Log: Cast #','Sample Log: Niskin #'])

In [ ]:
result.columns.values

In [43]:
sal_unit = '[' + list(set(sal_df['SAL: Unit']) )[0] + ']'
oxy_unit = '[' + list(set(oxy_df['OXY: Unit']) )[0] + ']'
sal_unit, oxy_unit

('[psu]', '[mL/L]')

In [44]:
result.drop(['Sample Log: Cast #','Sample Log: Niskin #'], axis=1, inplace=True)

In [45]:
result.rename(columns=lambda x: x.replace('SAL:','Bottle') + ' ' + sal_unit if 'SAL:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('OXY:','Bottle') + ' ' + oxy_unit if 'OXY:' in x else x, inplace=True)

In [49]:
result

,Niskin #,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,"Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Hex name,Start Latitude,Start Longitude,Start Time,Cast #,Bottle Salinity [psu],Bottle Oxy [mL/L]
0,1,Jul 06 2017 23:39:38,305.424,302.863,39.88712,-71.36012,8.2413,8.2419,3.672995,3.670923,...,6.55704,0.1732,95.7628 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
1,2,Jul 06 2017 23:43:52,280.064,277.733,39.88712,-71.36012,8.6133,8.6153,3.704244,3.706693,...,6.50402,0.1719,95.7945 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
2,3,Jul 06 2017 23:46:53,239.547,237.577,39.88712,-71.36014,9.9069,9.9174,3.827218,3.838110,...,6.32078,0.1700,95.8381 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
3,4,Jul 06 2017 23:49:19,204.555,202.889,39.88712,-71.36014,10.6219,10.6233,3.911485,3.911304,...,6.21697,0.1680,95.8875 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
4,5,Jul 06 2017 23:51:56,179.138,177.691,39.88714,-71.36014,11.2733,11.2787,3.980344,3.980555,...,6.12850,0.1667,95.9183 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
5,6,Jul 06 2017 23:56:09,138.845,137.736,39.88710,-71.36012,12.1869,12.1882,4.074497,4.074299,...,6.00934,0.1664,95.9258 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
6,7,Jul 06 2017 23:59:11,103.737,102.918,39.88710,-71.36014,12.4195,12.4085,4.090826,4.089482,...,5.98209,0.1683,95.8786 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
7,8,Jul 07 2017 00:01:54,78.658,78.041,39.88712,-71.36012,12.3134,12.3131,4.064800,4.064373,...,6.00061,0.1742,95.7391 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
8,9,Jul 07 2017 00:05:17,53.622,53.205,39.88714,-71.36013,12.0988,12.1000,4.005337,4.005199,...,6.04139,0.2120,94.8379 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN
9,10,Jul 07 2017 00:07:52,38.593,38.294,39.88713,-71.36012,12.4680,12.4705,4.009986,4.009991,...,6.00613,0.2811,93.2137 (avg),D:\Data\ar20001.hex,39 53.23 N,071 21.61 W,2017-07-06T23:27:03Z,999,NaN,NaN


In [53]:
result.to_csv(dirpath+'Irminger-4_Summary.csv')

In [58]:
# Now, need to include the nutrient bottle data
os.listdir('Irminger/Irminger-4/Water Sampling')

['.~lock.Irminger_Sea-04_AR21_Nutrients_Sample_Data_2017-09-20_ver_1-00.xlsx#',
 'Irminger_Sea-04_AR21_CTD_Sampling_Log.xlsx',
 'Irminger_Sea-04_AR21_CTD_Sampling_Logs_written.pdf',
 'Irminger_Sea-04_AR21_Nutrients_Sample_Data_2017-09-20_ver_1-00.xlsx',
 'Irminger_Sea-04_AR21_Oxygen_and_Salinity_Sample_Data']

In [70]:
nutrients = pd.read_excel('Irminger/Irminger-4/Water Sampling/Irminger_Sea-04_AR21_Nutrients_Sample_Data_2017-09-20_ver_1-00.xlsx',header=0)

In [71]:
nutrients.columns.values

array(['Sample ID', 'Cruise', 'Avg: Nitrate + Nitrite [µmol/L]',
       'Avg: Ammonium [µmol/L]', 'Avg: Phosphate [µmol/L]',
       'Avg: Silicate [µmol/L]', 'Avg: Nitrite [µmol/L]',
       'Avg: Nitrate [µmol/L]'], dtype=object)

In [72]:
nutrients

,Sample ID,Cruise,Avg: Nitrate + Nitrite [µmol/L],Avg: Ammonium [µmol/L],Avg: Phosphate [µmol/L],Avg: Silicate [µmol/L],Avg: Nitrite [µmol/L],Avg: Nitrate [µmol/L]
0,7-1,AR21,8.202829,1.1160,0.597241,5.578061,<0.04,8.162829
1,7-2,AR21,7.356260,1.1155,0.537812,4.344126,<0.04,7.316260
2,7-3,AR21,8.742555,1.2600,0.588400,5.171327,<0.04,8.702555
3,7-4,AR21,9.456192,1.0390,0.548126,5.188230,<0.04,9.416192
4,7-5,AR21,4.356785,0.8490,0.282904,1.024757,0.103333,4.253451
5,7-6,AR21,4.879019,0.6785,0.292236,1.239217,0.0713333,4.807686
6,12-1,AR21,5.594156,1.5190,0.520621,3.523264,<0.04,5.554156
7,12-2,AR21,7.290293,1.2500,0.628675,4.280739,0.0453333,7.244960
8,12-3,AR21,7.546163,1.3875,0.570228,4.156606,<0.04,7.506163
9,12-4,AR21,15.878926,1.0040,0.831029,6.879608,0.151333,15.727593


In [73]:
result.columns.values

array(['Niskin #', 'Date Time', 'Pressure, Digiquartz [db]',
       'Depth [salt water, m]', 'Latitude [deg]', 'Longitude [deg]',
       'Temperature [ITS-90, deg C]', 'Temperature, 2 [ITS-90, deg C]',
       'Conductivity [S/m]', 'Conductivity, 2 [S/m]',
       'Salinity, Practical [PSU]', 'Salinity, Practical, 2 [PSU]',
       'Oxygen, SBE 43 [ml/l]',
       'Oxygen Saturation, Garcia & Gordon [ml/l]',
       'Beam Attenuation, WET Labs C-Star [1/m]',
       'Beam Transmission, WET Labs C-Star [%]', 'Hex name',
       'Start Latitude', 'Start Longitude', 'Start Time', 'Cast #',
       'Bottle Salinity [psu]', 'Bottle Oxy [mL/L]'], dtype=object)

In [74]:
sample_log.columns.values

array(['Sample Log: Cast #', 'Sample Log: Niskin #', 'Sample Log: Time',
       'Sample Log: Target Depth', 'Sample Log: Trip Depth',
       'Sample Log: Potential Temp', 'Sample Log: Salinity',
       'Sample Log: Oxygen Bottle #', 'Sample Log: Ph Bottle #',
       'Sample Log: DIC/TA Bottle #', 'Sample Log: Salts Bottle #',
       'Sample Log: Nitrate Bottle 1',
       'Sample Log: Chlorophyll Brown Bottle #',
       'Sample Log: Chlorophyll Filter Sample #',
       'Sample Log: Unnamed: 14',
       'Sample Log: Chlorophyll Brown Bottle Volume',
       'Sample Log: Chlorophyll LN Tube', 'Sample Log: Comments',
       'SAL: Cruise', 'SAL: Station', 'SAL: Niskin #', 'SAL: Case ID',
       'SAL: Sample', 'SAL: Salinity', 'SAL: Unit', 'OXY: Cruise',
       'OXY: Station', 'OXY: Niskin #', 'OXY: Case', 'OXY: Sample#',
       'OXY: Oxy', 'OXY: Unit'], dtype=object)

In [84]:
nutrient_bottles = sample_log['Sample Log: Nitrate Bottle 1'].str.split('\n').apply(pd.Series, 1).stack()
nutrient_bottles.index = nutrient_bottles.index.droplevel(-1)
nutrient_bottles.name = 'Nitrate Bottle #'

In [88]:
sample_log = sample_log.join(nutrient_bottles)

In [92]:
sample_log['Nitrate Bottle #'].iloc[5]

'7-1.'

In [94]:
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: str(x).replace('.','') )

In [95]:
# Now I can add the nutrient bottle data to the sample log before loading
sample_log.merge(nutrients, how='left', left_on='Nitrate Bottle #', right_on='Sample ID')

,Sample Log: Cast #,Sample Log: Niskin #,Sample Log: Time,Sample Log: Target Depth,Sample Log: Trip Depth,Sample Log: Potential Temp,Sample Log: Salinity,Sample Log: Oxygen Bottle #,Sample Log: Ph Bottle #,Sample Log: DIC/TA Bottle #,...,OXY: Unit,Nitrate Bottle #,Sample ID,Cruise,Avg: Nitrate + Nitrite [µmol/L],Avg: Ammonium [µmol/L],Avg: Phosphate [µmol/L],Avg: Silicate [µmol/L],Avg: Nitrite [µmol/L],Avg: Nitrate [µmol/L]
0,006,1.0,NaN,NaN,1000.0,NaN,34.86,NaN,NaN,NaN,...,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,006,2.0,NaN,NaN,800.0,NaN,34.86,NaN,NaN,NaN,...,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,006,3.0,NaN,NaN,800.0,NaN,34.86,NaN,NaN,NaN,...,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,006,4.0,NaN,NaN,300.0,NaN,34.86,NaN,NaN,NaN,...,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,006,5.0,NaN,NaN,300.0,NaN,34.86,NaN,NaN,NaN,...,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,007,1.0,20:51:00,1500.0,1500.0,1.99604,34.90,T1,NaN,NaN,...,mL/L,7-1,7-1,AR21,8.202829,1.1160,0.597241,5.578061,<0.04,8.162829
6,007,2.0,21:03:00,1000.0,1000.0,NaN,34.85,T2,NaN,NaN,...,mL/L,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,007,3.0,21:12:00,500.0,500.0,NaN,34.86,T3,NaN,NaN,...,mL/L,7-2,7-2,AR21,7.356260,1.1155,0.537812,4.344126,<0.04,7.316260
8,007,4.0,21:13:00,480.0,480.0,NaN,34.88,T4,NaN,NaN,...,mL/L,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,007,5.0,21:19:00,250.0,250.0,NaN,34.90,T5,NaN,NaN,...,mL/L,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
nutrients

,Sample ID,Cruise,Avg: Nitrate + Nitrite [µmol/L],Avg: Ammonium [µmol/L],Avg: Phosphate [µmol/L],Avg: Silicate [µmol/L],Avg: Nitrite [µmol/L],Avg: Nitrate [µmol/L]
0,7-1,AR21,8.202829,1.1160,0.597241,5.578061,<0.04,8.162829
1,7-2,AR21,7.356260,1.1155,0.537812,4.344126,<0.04,7.316260
2,7-3,AR21,8.742555,1.2600,0.588400,5.171327,<0.04,8.702555
3,7-4,AR21,9.456192,1.0390,0.548126,5.188230,<0.04,9.416192
4,7-5,AR21,4.356785,0.8490,0.282904,1.024757,0.103333,4.253451
5,7-6,AR21,4.879019,0.6785,0.292236,1.239217,0.0713333,4.807686
6,12-1,AR21,5.594156,1.5190,0.520621,3.523264,<0.04,5.554156
7,12-2,AR21,7.290293,1.2500,0.628675,4.280739,0.0453333,7.244960
8,12-3,AR21,7.546163,1.3875,0.570228,4.156606,<0.04,7.506163
9,12-4,AR21,15.878926,1.0040,0.831029,6.879608,0.151333,15.727593


In [80]:
A.index = A.index.droplevel(-1)
A

5      7-1.
7      7-2.
11     7-3.
11     7-4.
15     7-5.
15     7-6.
16     7-7.
53    12-1.
57    12-2.
58    12-3.
59    12-4.
61    12-5.
62    12-6.
63    12-7.
dtype: object

In [81]:
A1 = sample_log

In [82]:
A.name = 'Nitrate Bottle #'

In [83]:
A1.join(A)

,Sample Log: Cast #,Sample Log: Niskin #,Sample Log: Time,Sample Log: Target Depth,Sample Log: Trip Depth,Sample Log: Potential Temp,Sample Log: Salinity,Sample Log: Oxygen Bottle #,Sample Log: Ph Bottle #,Sample Log: DIC/TA Bottle #,...,SAL: Salinity,SAL: Unit,OXY: Cruise,OXY: Station,OXY: Niskin #,OXY: Case,OXY: Sample#,OXY: Oxy,OXY: Unit,Nitrate Bottle #
0,006,1.0,NaN,NaN,1000.0,NaN,34.86,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,006,2.0,NaN,NaN,800.0,NaN,34.86,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,006,3.0,NaN,NaN,800.0,NaN,34.86,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,006,4.0,NaN,NaN,300.0,NaN,34.86,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,006,5.0,NaN,NaN,300.0,NaN,34.86,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,007,1.0,20:51:00,1500.0,1500.0,1.99604,34.90,T1,NaN,NaN,...,34.9281,psu,AR21,007,1,T,T1,6.445,mL/L,7-1.
6,007,2.0,21:03:00,1000.0,1000.0,NaN,34.85,T2,NaN,NaN,...,34.8731,psu,AR21,007,2,T,T2,6.927,mL/L,NaN
7,007,3.0,21:12:00,500.0,500.0,NaN,34.86,T3,NaN,NaN,...,34.8958,psu,AR21,007,3,T,T3,6.872,mL/L,7-2.
8,007,4.0,21:13:00,480.0,480.0,NaN,34.88,T4,NaN,NaN,...,34.8954,psu,AR21,007,4,T,T4,6.917,mL/L,NaN
9,007,5.0,21:19:00,250.0,250.0,NaN,34.90,T5,NaN,NaN,...,34.9111,psu,AR21,007,5,T,T5,6.785,mL/L,NaN
